In [61]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import namedtuple
from math import sqrt
import os

import tensorflow as tf

In [2]:
batch_norm = tf.contrib.layers.batch_norm
convolution2d = tf.contrib.layers.convolution2d

In [68]:
def res_net(x, y, activation=tf.nn.relu):
    """Builds a residual network.
  Note that if the input tensor is 2D, it must be square in order to be
  converted to a 4D tensor.
  Borrowed structure from:
  github.com/pkmital/tensorflow_tutorials/blob/master/10_residual_network.py
  Args:
    x: Input of the network
    y: Output of the network
    activation: Activation function to apply after each convolution
  Returns:
    Predictions and loss tensors.
      """

  # Configurations for each bottleneck group.
    BottleneckGroup = namedtuple('BottleneckGroup',
                               ['num_blocks', 'num_filters', 'bottleneck_size'])
    groups = [
      BottleneckGroup(3, 128, 32), BottleneckGroup(3, 256, 64),
      BottleneckGroup(3, 512, 128), BottleneckGroup(3, 1024, 256)
    ]
    
    input_shape = x.get_shape().as_list()

    # Reshape the input into the right shape if it's 2D tensor
    if len(input_shape) == 2:
        ndim = int(sqrt(input_shape[1]))
        x = tf.reshape(x, [-1, ndim, ndim, 1])

    # First convolution expands to 64 channels
    with tf.variable_scope('conv_layer1'):
        net = convolution2d(
            x, 64, 7, normalizer_fn=batch_norm, activation_fn=activation)
    # Max pool
    net = tf.nn.max_pool(net, [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # First chain of resnets
    with tf.variable_scope('conv_layer2'):
        net = convolution2d(net, groups[0].num_filters, 1, padding='VALID')
        
    # Create the bottleneck groups, each of which contains `num_blocks`
    # bottleneck groups.
    for group_i, group in enumerate(groups):
        for block_i in range(group.num_blocks):
            name = 'group_%d/block_%d' % (group_i, block_i)
            
            # 1x1 convolution responsible for reducing dimension
            with tf.variable_scope(name + '/conv_in'):
                conv = convolution2d(
                    net,
                    group.bottleneck_size,
                    1,
                    padding='VALID',
                    activation_fn=activation,
                    normalizer_fn=batch_norm)
                
            with tf.variable_scope(name + '/conv_bottleneck'):
                conv = convolution2d(
                    conv,
                    group.bottleneck_size,
                    3,
                    padding='SAME',
                    activation_fn=activation,
                    normalizer_fn=batch_norm)

            # 1x1 convolution responsible for restoring dimension
            with tf.variable_scope(name + '/conv_out'):
                input_dim = net.get_shape()[-1].value
                conv = convolution2d(
                    conv,
                    input_dim,
                    1,
                    padding='VALID',
                    activation_fn=activation,
                    normalizer_fn=batch_norm)

              # shortcut connections that turn the network into its counterpart
              # residual function (identity shortcut)
            net = conv + net

        try:
            next_group = groups[group_i + 1]
            with tf.variable_scope('block_%d/conv_upscale' % group_i):
                net = convolution2d(
                    net,
                    next_group.num_filters,
                    1,
                    activation_fn=None,
                    biases_initializer=None,
                    padding='SAME')
        except IndexError:
            pass

    net_shape = net.get_shape().as_list()
    net = tf.nn.avg_pool(
        net,
        ksize=[1, net_shape[1], net_shape[2], 1],
        strides=[1, 1, 1, 1],
        padding='VALID')
    
    net_shape = net.get_shape().as_list()
    net = tf.reshape(net, [-1, net_shape[1] * net_shape[2] * net_shape[3]])
    
    target = tf.one_hot(y, depth=10, dtype=tf.float32)
    logits = tf.contrib.layers.fully_connected(net, 10, activation_fn=None)
    loss = tf.losses.softmax_cross_entropy(target, logits)
    
    return tf.nn.softmax(logits), loss
    
def res_net_model(x, y):
    prediction, loss = res_net(x, y)
    predicted = tf.argmax(prediction, 1)
    accuracy = tf.equal(predicted, tf.cast(y, tf.int64))
    predictions = {'prob': prediction, 'class': predicted, 'accuracy': accuracy}

    train_op = tf.contrib.layers.optimize_loss(
      loss,
      tf.contrib.framework.get_global_step(),
      optimizer='Adagrad',
      learning_rate=0.001)
    
    return predictions, loss, train_op

training model

In [73]:
# Download and load MNIST data.
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

# Create a new resnet classifier.
classifier = tf.contrib.learn.Estimator(model_fn=res_net_model, model_dir='./model')

tf.logging.set_verbosity(tf.logging.INFO) # Show training logs

# Train model and save summaries into logdir.
classifier.fit(mnist.train.images, mnist.train.labels, batch_size=100, steps=1000)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_environment': 'local', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000250592B0>, '_evaluation_master': '', '_save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_master': ''}


INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_environment': 'local', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000250592B0>, '_evaluation_master': '', '_save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_master': ''}


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


...

INFO:tensorflow:Saving checkpoints for 1000 into ./model\model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into ./model\model.ckpt.


INFO:tensorflow:Loss for final step: 0.100811.


INFO:tensorflow:Loss for final step: 0.100811.


Estimator(params=None)

test model

In [71]:
batch_size = 10
batch_num = int(mnist.test.num_examples / batch_size)
result = []
score = 0

In [64]:
# Calculate accuracy.(test)
for i in range(100):
    batch = mnist.test.next_batch(batch_size)
    
    result.append(
        classifier.evaluate(
        x=batch[0],
        y=batch[1],
        metrics= {
            'accuracy' :
                tf.contrib.learn.MetricSpec(
                    metric_fn = tf.contrib.metrics.streaming_accuracy,
                    prediction_key='accuracy'
                ),
        }
        )
    )
    
print(len(result))
# for res in result:
#     score += res['accuracy']

# print('Accuracy : ', score/batch_num)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


INFO:tensorflow:Starting evaluation at 2017-05-04-11:11:18


...

INFO:tensorflow:Finished evaluation at 2017-05-04-11:12:15


INFO:tensorflow:Finished evaluation at 2017-05-04-11:12:15


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.08, global_step = 1000, loss = 0.126942


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.08, global_step = 1000, loss = 0.126942


549


In [67]:
# Calculate accuracy.
result = classifier.evaluate(
    x=mnist.test.images[:200],
    y=mnist.test.labels[:200],
    metrics={
        'accuracy':
            tf.contrib.learn.MetricSpec(
                metric_fn=tf.contrib.metrics.streaming_accuracy,
                prediction_key='accuracy'),
    })
score = result['accuracy']
print('Accuracy: {0:f}'.format(score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


C:\Users\Administrator\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


INFO:tensorflow:Starting evaluation at 2017-05-04-11:17:59


...

INFO:tensorflow:Finished evaluation at 2017-05-04-11:18:42


INFO:tensorflow:Finished evaluation at 2017-05-04-11:18:42


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.115, global_step = 1000, loss = 0.173397


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.115, global_step = 1000, loss = 0.173397


Accuracy: 0.115000


In [65]:
result

[{'accuracy': 0.126, 'global_step': 1000, 'loss': 0.14870425},
 {'accuracy': 0.107, 'global_step': 1000, 'loss': 0.17997958},
 {'accuracy': 0.107, 'global_step': 1000, 'loss': 0.21735755},
 {'accuracy': 0.111, 'global_step': 1000, 'loss': 0.19259848},
 {'accuracy': 0.121, 'global_step': 1000, 'loss': 0.18864594},
 {'accuracy': 0.119, 'global_step': 1000, 'loss': 0.10162238},
 {'accuracy': 0.113, 'global_step': 1000, 'loss': 0.13720106},
 {'accuracy': 0.112, 'global_step': 1000, 'loss': 0.084695332},
 {'accuracy': 0.113, 'global_step': 1000, 'loss': 0.081935704},
 {'accuracy': 0.121, 'global_step': 1000, 'loss': 0.11734874},
 {'global_step': 1000, 'loss': 0.10788562},
 {'global_step': 1000, 'loss': 0.11676155},
 {'global_step': 1000, 'loss': 0.24755821},
 {'global_step': 1000, 'loss': 0.2833387},
 {'global_step': 1000, 'loss': 0.075475737},
 {'global_step': 1000, 'loss': 0.069856942},
 {'global_step': 1000, 'loss': 0.10123716},
 {'global_step': 1000, 'loss': 0.09039817},
 {'global_step'

In [50]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [53]:
batch[0].shape[0]

1000